In [ ]:
import requests

html_filename = "Guten-Rutsch-und-ein-gesundes-neues-Jahr-2021-5001311.html"
r = requests.get(f"https://www.heise.de/news/{html_filename}")
open(html_filename, 'wb').write(r.content)

In [ ]:
html = open(html_filename).read()

In [ ]:
!pip install bs4

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html)

In [ ]:
soup.title

In [ ]:
soup.h1

In [ ]:
soup.h1.text

In [ ]:
soup.h1.text.strip()

In [ ]:
soup.select("#meldung > div.article-layout__header-container > header > h1")

In [ ]:
soup.select_one("#meldung > div.article-layout__header-container > header > h1")

In [ ]:
soup.select_one("#meldung > div.article-layout__header-container > header > p")

In [ ]:
soup.select("#meldung > div.article-layout__content-container > div > p")

In [ ]:
soup.select_one("a.redakteurskuerzel__link")

In [ ]:
soup.select_one("script[type=application/ld+json]")

In [ ]:
soup.find("script", type="application/ld+json")

In [ ]:
import json
ld = json.loads(soup.find("script", type="application/ld+json").string)
ld

In [ ]:
ld['datePublished']